In [ ]:
import os
import torchaudio
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim

# Путь к папке с чистыми аудиофайлами
clean_files = "/content/drive/MyDrive/clean_dataset/clean_drilling"

# Путь к папке с зашумленными аудиофайлами
noisy_files = "/content/drive/MyDrive/noisy_dataset/noisy_drilling"

# Класс для загрузки и предобработки аудиофайлов
class AudioDataset(Dataset):
    def __init__(self, clean_files, noisy_files):
        self.clean_files = clean_files
        self.noisy_files = noisy_files
        self.clean_audio = self.load_audio_files(clean_files)
        self.noisy_audio = self.load_audio_files(noisy_files)

    def load_audio_files(self, files):
        audio_data = []
        for file in os.listdir(files):
            file_path = os.path.join(files, file)
            waveform, _ = torchaudio.load(file_path)
            audio_data.append(waveform)
        return audio_data

    def __len__(self):
        return len(self.clean_audio)

    def __getitem__(self, idx):
        clean_wav = self.clean_audio[idx]
        noisy_wav = self.noisy_audio[idx]
        return clean_wav, noisy_wav


In [ ]:
full_dataset = AudioDataset(clean_files, noisy_files)
import torch
# Разделение на обучающий и тестовый наборы
train_dataset = torch.utils.data.Subset(full_dataset, range(400))
test_dataset = torch.utils.data.Subset(full_dataset, range(400, len(full_dataset)))

input_size = 2  # У нас есть два канала в аудио
hidden_size = 64  # Размер скрытого слоя
output_size = 2  # Столько же каналов на выходе, сколько и на входе


In [ ]:
class DenoisingAutoencoder(nn.Module):
    def __init__(self):
        super(DenoisingAutoencoder, self).__init__()
        # Encoder
        self.encoder = nn.Sequential(
            nn.Conv1d(in_channels=2, out_channels=16, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv1d(in_channels=16, out_channels=8, kernel_size=3, padding=1),
            nn.ReLU()
        )
        # Decoder
        self.decoder = nn.Sequential(
            nn.ConvTranspose1d(in_channels=8, out_channels=16, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.ConvTranspose1d(in_channels=16, out_channels=2, kernel_size=3, padding=1),
            nn.Sigmoid()  # Чтобы значения были в диапазоне [0, 1]
        )

    def forward(self, x):
        # Прямой проход
        x = self.encoder(x)
        x = self.decoder(x)
        return x

# Создаем экземпляр модели
model = DenoisingAutoencoder()

# Функция потерь и оптимизатор
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Обучение
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
num_epochs = 8
for epoch in range(num_epochs):
    print(f"Epoch {epoch+1}/{num_epochs}")
    running_loss = 0.0
    for clean_wav, noisy_wav in train_loader:
        optimizer.zero_grad()
        outputs = model(noisy_wav)
        loss = criterion(outputs, clean_wav)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f"Loss: {running_loss}")


In [ ]:
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)
test_loss = 0.0
model.eval()  # Перевод модели в режим оценки
with torch.no_grad():
    for clean_wav, noisy_wav in test_loader:
        outputs = model(noisy_wav)
        loss = criterion(outputs, clean_wav)
        test_loss += loss.item()

print(f"Test Loss: {test_loss}")

In [ ]:
import os
import torch
import torchaudio
import matplotlib.pyplot as plt
'''
# Путь к папке с зашумленными аудиофайлами
noisy_files = "/content/drive/MyDrive/noisy_dataset/noisy_car_horn"
clean_files = "/content/drive/MyDrive/clean_dataset/clean_car_horn"

# Загрузка модели
model = DenoisingAutoencoder()
model.load_state_dict(torch.load('denoising_model.pth'))
model.eval()'''

# Загрузка одного зашумленного аудиофайла
clean_file_name = "/content/drive/MyDrive/clean_dataset/clean_drilling/common_voice_en_36997166.wav"  # замените на имя файла, который хотите использовать
noisy_file_name = "/content/drive/MyDrive/noisy_dataset/noisy_drilling/common_voice_en_36997166.wav"  # замените на имя файла, который хотите использовать
noisy_path = os.path.join(noisy_files, noisy_file_name)
clean_path = os.path.join(clean_files, clean_file_name)
noisy_signal, sr = torchaudio.load(noisy_path)
clean_signal, sr = torchaudio.load(clean_path)

# Применение модели к зашумленному аудиосигналу
with torch.no_grad():
    denoised_signal = model(noisy_signal)

# Сохранение отфильтрованного аудиофайла
output_file = '/content/denoised_example.wav'
torchaudio.save(output_file, denoised_signal, sr)

# Визуализация сигналов
def plot_waveforms(noisy, denoised, clean, sr):
    fig, axs = plt.subplots(3, 1, figsize=(15, 10))
    axs[0].plot(noisy.t().numpy())
    axs[0].set_title('Noisy Signal')
    axs[1].plot(denoised.t().numpy())
    axs[1].set_title('Denoised Signal')
    axs[2].plot(clean.t().numpy())
    axs[2].set_title('Clean Signal')
    for ax in axs:
        ax.set_xlabel('Samples')
        ax.set_ylabel('Amplitude')
    plt.tight_layout()
    plt.show()

plot_waveforms(noisy_signal, denoised_signal, clean_signal, sr)

# Прослушивание аудиофайлов (только если работаете в среде, поддерживающей аудиовывод)
from IPython.display import Audio
print("Noisy Signal:")
display(Audio(noisy_path))
print("Denoised Signal:")
display(Audio(output_file))
print("Clean Signal:")
display(Audio(clean_path))
